In [2]:
import pandas as pd
import numpy as np
import pickle

import matplotlib.pyplot as plt
import seaborn as sns

# from sklearn.tree import DecisionTreeClassifier, plot_tree
# from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
# import lightgbm as lgb
# from catboost import CatBoostClassifier

import warnings
warnings.filterwarnings('ignore')

### Models

In [9]:
rf = 'modeling/rf.pkl'
gbm = 'modeling/gbm.pkl'
lgb = 'modeling/lgb.pkl'
catboost = 'modeling/catboost.pkl'

with open(rf, 'rb') as file:
    RF = pickle.load(file)
    
with open(gbm, 'rb') as file:
    GBM = pickle.load(file)
    
with open(lgb, 'rb') as file:
    LGB = pickle.load(file)
    
with open(catboost, 'rb') as file:
    CATBOOST= pickle.load(file)

### Data (in necessary format)

In [4]:
data = pd.read_csv('data/scraped_for_modeling_labeled.csv')
data.fillna(0, inplace = True) # divisions with 0

X = data.drop(['winner', 'played'], 1).copy()
y = data['played'].copy()

In [6]:
# from boruta elimination method:
keep_cols = ['offense_downs_Fourth Downs_PCT', 'offense_passing_AVG', 'offense_passing_YDS/G', 'offense_passing_RTG', 
             'offense_receiving_AVG', 'defense_passing_AVG', 'defense_passing_RTG', 'defense_receiving_AVG', 
             'punting_punts_per_game', 'defense_passing_SYL_per_game', 'offense_downs_Third Downs_ATT_per_game', 
             'defense_downs_Third Downs_ATT_per_game', 'defense_downs_Fourth Downs_ATT_per_game', 
             'defense_downs_First Downs_penalty_ratio', 'offense_passing_TD_per_game', 
             'offense_passing_INT_per_game', 'defense_passing_INT_per_game', 'offense_pass_TD_to_INT', 
             'defense_pass_TD_to_INT']

X = X[['Team', 'games_played', 'season'] + keep_cols].copy()

### Prediction

In [18]:
predictions = data[['Team', 'season', 'winner', 'played']].copy()

predictions['pred_rf'] = RF.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_gbm'] = GBM.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_lgb'] = LGB.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_catboost'] = CATBOOST.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_mean'] = (predictions['pred_rf'] + predictions['pred_gbm'] + predictions['pred_lgb'] + predictions['pred_catboost']) / 4

predictions.head(3)

,Team,season,winner,played,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean
0,Kansas City Chiefs,2004,0,0,0.096406,0.073115,0.054210,0.240884,0.116154
1,Indianapolis Colts,2004,0,0,0.352287,0.325699,0.204493,0.348993,0.307868
2,Green Bay Packers,2004,0,0,0.052101,0.040946,0.008661,0.226210,0.081979


### Compare predictions vs actuals

In [19]:
## todo